# End to End Tweet Sentiment analysis using TFIDF
In this notebook, we will be building a machine learning model to perform tweet sentiment analysis using pyspark. We will not be just training a simple model but we will also be exploring the use of advance functions in pyspark such as pipeline, gridsearch. Furthermore, we will me tracking our experiments using MLFOW and then deploying the model.


### Data
- We will be using Sentiment140 from standford http://help.sentiment140.com/for-students/
- I have reduced the size by 8 times to reduce the training time

### Data preparation
- pyspark inbuild pipeline 

### Model training
- pyspark inbuild crossvalidation
- pyspark inbuild gridsearch
- pyspark inbuild machine learning models

### Model Tracking
- MLFLOW


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count, countDistinct
from pyspark.ml.feature import Tokenizer, NGram, CountVectorizer, IDF, VectorAssembler
from pyspark.ml.classification import LogisticRegression, LogisticRegression
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.util import MLWriter


#mlflow
import mlflow
from mlflow import log_metric, log_param, log_artifacts

#python
import os

## Main config

In [2]:
class CONFIG:
    experiment_name='Sentiment_analysis_LR'
    mlflow_uri = "http://localhost:5000"
    raw_data_fp = os.path.join("..","data","sentiment_analysis.csv")
    

## Setting mlflow experiment name

In [3]:
mlflow.set_experiment(CONFIG.experiment_name)
mlflow.set_tracking_uri(CONFIG.mlflow_uri)

In [4]:
spark = SparkSession.builder.appName(CONFIG.experiment_name).config("spark.driver.memory", "10g").getOrCreate()

In [5]:
df = spark.read.format("csv").load(CONFIG.raw_data_fp,header=True, inferSchema=True)
df.show(5)

+------+---+----------+--------------------+--------+-------------+--------------------+
|   _c0|  0|         1|                   2|       3|            4|                   5|
+------+---+----------+--------------------+--------+-------------+--------------------+
|779483|  0|2322939033|Wed Jun 24 23:34:...|NO_QUERY|Olivia_Hebert|right about now, ...|
| 74208|  0|1694744393|Mon May 04 03:57:...|NO_QUERY|   MasqueArts|this rain wasn't ...|
|310272|  0|2001072003|Mon Jun 01 23:45:...|NO_QUERY|     phlthy01|@madathena: K nev...|
|290810|  0|1995500009|Mon Jun 01 13:35:...|NO_QUERY| FleaFletcher| there's so much ...|
|710133|  0|2257822485|Sat Jun 20 14:57:...|NO_QUERY|bikerchick250|I wish I were at ...|
+------+---+----------+--------------------+--------+-------------+--------------------+
only showing top 5 rows



## Changing the column names

In [6]:
new_column_names = ['index','polarity', 'id', 'date', 'query', 'user', 'text']
df = df.select([df[original].alias(new) for original, new in zip(df.columns,new_column_names)])
df.show(3)

+------+--------+----------+--------------------+--------+-------------+--------------------+
| index|polarity|        id|                date|   query|         user|                text|
+------+--------+----------+--------------------+--------+-------------+--------------------+
|779483|       0|2322939033|Wed Jun 24 23:34:...|NO_QUERY|Olivia_Hebert|right about now, ...|
| 74208|       0|1694744393|Mon May 04 03:57:...|NO_QUERY|   MasqueArts|this rain wasn't ...|
|310272|       0|2001072003|Mon Jun 01 23:45:...|NO_QUERY|     phlthy01|@madathena: K nev...|
+------+--------+----------+--------------------+--------+-------------+--------------------+
only showing top 3 rows



In [7]:
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- polarity: integer (nullable = true)
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- query: string (nullable = true)
 |-- user: string (nullable = true)
 |-- text: string (nullable = true)



In [8]:
df.describe().show()

+-------+------------------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|summary|             index|          polarity|                  id|                date|   query|                user|                text|
+-------+------------------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|  count|            200000|            200000|              200000|              200000|  200000|              200000|              200000|
|   mean|      800244.80057|               2.0|  1.99898937403071E9|                null|    null|1.3452843115681818E9|                null|
| stddev|462222.90411669045|2.0000050000187506|1.9375194913178074E8|                null|    null| 8.371814238571698E9|                null|
|    min|                11|                 0|          1467812579|Fri Apr 17 20:30:...|NO_QUERY|        000catnap000|                 ...|
|    max|    

We can see that we have 200k rows of tweets which is quite a huge number of data.

In [9]:
df.select([count(when(isnan(col),col)).alias('missing_'+col) for col in df.columns]).show(5)

+-------------+----------------+----------+------------+-------------+------------+------------+
|missing_index|missing_polarity|missing_id|missing_date|missing_query|missing_user|missing_text|
+-------------+----------------+----------+------------+-------------+------------+------------+
|            0|               0|         0|           0|            0|           0|           0|
+-------------+----------------+----------+------------+-------------+------------+------------+



Fortunately for us, we do not have any missing values. We can also see that for pyspark, checking of missing values is not as straight forward in pandas where we just  have to use isna().sum()

## Value Counts
Although we will be using only the tweets as an input to the model, we do want to do some sanity check on our dataset and this also important for continuous training to identify drift in the input data

1. we can see that there are no neutral tweets, all of the tweets are either 0(negative) or 4(positive), we will change the polarity to [0,1] column and frame the problem statment into a binary classifier
2. The dataset is very balanced with a equal split between the positive and negative tweets
3. All of the columns are no query and therefore we can drop that column since there are no variation

In [10]:
for column in ['polarity','query','user']:
    print(f"For column {column}")
    df.select(column).groupby(column).agg(count(column).alias("count")).orderBy(col("count").desc()).show()

For column polarity
+--------+------+
|polarity| count|
+--------+------+
|       0|100000|
|       4|100000|
+--------+------+

For column query
+--------+------+
|   query| count|
+--------+------+
|NO_QUERY|200000|
+--------+------+

For column user
+---------------+-----+
|           user|count|
+---------------+-----+
|       lost_dog|   62|
|        webwoke|   45|
|       tweetpet|   43|
|SallytheShizzle|   40|
|    VioletsCRUK|   40|
|      Jayme1988|   39|
|      DarkPiano|   35|
|    mcraddictal|   35|
|    Karen230683|   35|
|    what_bugs_u|   34|
|   SongoftheOss|   34|
|         keza34|   34|
|       tsarnick|   31|
|     Spidersamm|   30|
|          StDAY|   30|
|   TraceyHewins|   29|
|     SarahSaner|   29|
|    Dutchrudder|   25|
|        Dogbook|   25|
|      shanajaca|   24|
+---------------+-----+
only showing top 20 rows



In [11]:
df = df.withColumn('label', when(df['polarity']==0, 0).otherwise(1)).drop('polarity')
column='label'
df.select(column).groupby(column).agg(count(column).alias("count")).orderBy(col("count").desc()).show()

+-----+------+
|label| count|
+-----+------+
|    1|100000|
|    0|100000|
+-----+------+



In [12]:
column = 'user'
df.select(countDistinct(column)).show()

+--------------------+
|count(DISTINCT user)|
+--------------------+
|              148659|
+--------------------+



In [13]:
full_train_df = df.select(['text','label'])
full_train_df.show(2)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|right about now, ...|    0|
|this rain wasn't ...|    0|
+--------------------+-----+
only showing top 2 rows



In [14]:
full_train_df = full_train_df.dropDuplicates(['text'])
full_train_df.describe().show()

+-------+--------------------+-----------------+
|summary|                text|            label|
+-------+--------------------+-----------------+
|  count|              199146|           199146|
|   mean|                null| 0.50034145802577|
| stddev|                null|0.500001138771227|
|    min|                 ...|                0|
|    max|ï¿½ï¿½ï¿½ï¿½ï¿½ß§...|                1|
+-------+--------------------+-----------------+



For huge dataset like this, it is costly to combine the model into the pipeline and train in cross-validated grid search and therefore for this project the transformation and the model is seperated

In [16]:
max_ngram = 3 #trigram
tokenizer = Tokenizer(inputCol='text', outputCol='tokenized')

n_gram_pipe = [NGram(n=n, inputCol='tokenized', outputCol=f"{n}_gram")
  for n in range(1, max_ngram+1) 
]
hash_tf_pipe = [CountVectorizer(vocabSize=12000, inputCol=f"{n}_gram", outputCol=f"{n}_gram_tf")
  for n in range(1, max_ngram+1)               
]
idf_pipe = [IDF(inputCol=f"{n}_gram_tf", outputCol=f"{n}_gram_idf", minDocFreq=5)
  for n in range(1, max_ngram+1)
]

vector_assembler = VectorAssembler(inputCols=[f"{n}_gram_idf" for n in range(1, max_ngram+1)], outputCol='features')

feature_transformation_pipeline = Pipeline(stages=[tokenizer, *n_gram_pipe, *hash_tf_pipe, *idf_pipe, vector_assembler])

In [17]:
fitted_pipeline = feature_transformation_pipeline.fit(full_train_df)
transformed_df = fitted_pipeline.transform(full_train_df)

In [18]:
transformed_df = transformed_df.select(['features', 'label'])

In [19]:
transformed_df.show(2)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(42000,[2,62,362,...|    0|
|(42000,[0,1,8,14,...|    0|
+--------------------+-----+
only showing top 2 rows



## Hyperparameter search
One down side of pyspark ml is the lack of stratifiedKfold split. For this project, we will be using TrainValidationSplit instead due to the long training time

In [20]:
mlflow.autolog()

2022/01/05 15:49:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2022/01/05 15:49:23 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for pyspark: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2022/01/05 15:49:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected

In [21]:
with mlflow.start_run() as active_run:
    model = LogisticRegression(featuresCol='features', labelCol='label')
    param_grid = ParamGridBuilder().addGrid(model.elasticNetParam,[0.1,0.15]).addGrid(model.regParam,[0.05,0.1]).build()

    train_valid_clf = TrainValidationSplit(estimator=model, 
                                           estimatorParamMaps=param_grid, 
                                           evaluator=BinaryClassificationEvaluator(labelCol='label'), 
                                           trainRatio=0.9)
    trained_model = train_valid_clf.fit(transformed_df)


2022/01/05 15:49:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2022/01/05 15:49:33 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2022/01/05 15:49:35 WARNING mlflow.utils.autologging_utils: Encountered unex

2022/01/05 16:34:17 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable


In [22]:
print(trained_model.validationMetrics)

[0.8295957362589691, 0.7916592198633501, 0.8092940953845127, 0.7573227502238503]
